 # MGF Filterer
 
 This scripts allows two things:
 
 1. Filter a raw mgf according to metadata present in individual blocks headers
         Note: this metadata file should contain a spectrum_id_val column
 2. filter mass spec data to keep the X most intense peaks of each spectra

In [2]:
%%time

###MGF Filterer
#This scripts allows two things:
#Filter a raw mgf according to metadata present in individual blocks headers
#Filter mass spec data to keep the X most intense peaks of each spectra and also keep only spectra with a minimal peak number



import re
import numpy as np
import pandas as pd



mgfdata = []

#the raw mgf we want to filter    
fh = open(r"/Users/pma/Dropbox/Research_UNIGE/Projets/Ongoing/Taxo_weigher/Taxo_Data_OSF/Full_GNPS_lib_filtered_top500.mgf", "r").read()

#the output mgf file
f= open("/Users/pma/Dropbox/Research_UNIGE/Projets/Ongoing/Taxo_weigher/Taxo_Data_OSF/Benchmark_dataset_spectral.mgf","w+")

#here we fix the topx (amount of most intense peaks) and minpeaks (minimal numbers of peak per spectra) values:

topx= 500
minpeaks= 6


#the metadata we want to filter according to (optional) NB: in case you want to filter according to an external file please uncomment line 99
df_MH = pd.read_csv("/Users/pma/Dropbox/Research_UNIGE/Projets/Ongoing/Taxo_weigher/Taxo_Data_OSF/Benchmark_dataset_metadata.tsv", sep='\t') 



contents = re.split(r"BEGIN IONS",fh)
contents.pop(0)


for item in contents:
    mgfdata_dict = {}
    
    #this is the regex to extract the spectrumid value
         
    s = re.compile(r'^SPECTRUMID=.*', re.M)

    spectrumid_f = re.search(s, item)

    ss = re.compile(r'(?:SPECTRUMID=)(.*)', re.M)
    
    if spectrumid_f is not None:
        spectrum_id = re.search(ss, spectrumid_f.group())
    else:
        spectrum_id = None

    if spectrum_id is not None:
        spectrum_id_val = spectrum_id.group(1)
    else:
        spectrum_id_val = None
      
    
    #this is the regex to extract the mass and intensitie values
    
    p = re.compile(r'^([0-9].*)\t([0-9].*)', re.M)

    mass_int = re.findall(p, item)

    #we keep mass and intensities separately as list 
    mass = [x[0] for x in mass_int]
    intensities = [x[1] for x in mass_int]
 
    #we then convert them to float and finally keep them as tupple
    
    mass = tuple([float(i) for i in mass])
    intensities = tuple([float(i) for i in intensities])

    #here we filter intensities for the x most intense and return the output. We also fix a minimal number of peaks at this step else empty spectra will return errors
    if len(mass) >= minpeaks:
        intensities,mass = zip(*sorted(zip(intensities,mass),reverse=True)[0:topx])
        mass,intensities = zip(*sorted(zip(mass,intensities)))
    
    #we turn them back to strings and zip them up
    
    mass = [str(i) for i in mass]
    intensities = [str(i) for i in intensities]
    
    zipped_masslist = list(zip(mass, intensities))

    #here we capture the rest of the block metadata. This regex will basically capture everything except what starts with digits
    
    d = re.compile(r'^[^(0-9)].*', re.M)

    meta = re.findall(d, item)
    
    #and remove the 'END IONS' strings
    
    metaf = meta[:-2]
    
    #we now keep this as a block
    
    block = "\n".join(metaf)
    

    #Comment one of both lines wether you just want to do a mass intensity filtering or 
    #if you also want to filter according to an external list
    
    if (spectrum_id_val in df_MH[['spectrum_id_val']].values and len(zipped_masslist) >= minpeaks) :
    #if  len(zipped_masslist) >= minpeaks :   
    
        #print('BEGIN IONS' + block)
        #for line in zipped_masslist:
            #print('\t'.join(line))
        #print('END IONS' + '\n')
        
        f.write('\nBEGIN IONS' + block + '\n')
        for line in zipped_masslist:
            f.write('\t'.join(line) + '\n')
        f.write('END IONS' + '\n')
        

f.close()

CPU times: user 1min 12s, sys: 950 ms, total: 1min 13s
Wall time: 1min 13s
